Import required Libraries

In [1]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import shutil

# importing necessary libraries
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing, model_selection

from imblearn.over_sampling import SMOTE, RandomOverSampler


from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

**Feature Exxtraction class**

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pyparsing import Word, hexnums, WordEnd, Optional, alphas, alphanums


class FeatureExtraction:
    def __init__(self, trainLabelsCSVPath):
        self.classes = pd.read_csv(trainLabelsCSVPath)

  #returns a data fram containing the frequency of the hexadecimal in the file
    def feature_extraction_bytes(self, folder_path):
        total_dict = []
        #loops through a list of all files in the folder
        for file in os.listdir(folder_path):
          #checks if the file has a byte extension
          #processes only byte files
            if file.find('bytes') != -1:
                # print('current file', file)
                file_path = os.path.join(folder_path, file)
                # get unique count
                uniq = {}
                #updates the dictionary uniq with the hexcode as keys and their frequency as values
                #reads the byte file
                with open(file_path) as f:
                    lines = f.read().splitlines()
                    f.close()
                    for line in lines:
                      #split each line by space
                      #the first item is ignored since it is not a hexadecimal
                        byt = line.split(' ')[1:]
                        #looping through the hexadecimals
                        for hexa in byt:
                            if len(hexa) > 1:
                                if hexa in uniq:
                                    uniq[hexa] += 1
                                else:
                                    uniq[hexa] = 1

                # get the class value 
                label = self.classes.loc[self.classes['Id'] == file.split('.')[0]]['Class'].values[0]
                uniq['Class'] = label
                # print(len(uniq))
                # print(uniq)
                total_dict.append(uniq)

                data = pd.DataFrame(total_dict)
        return data

    # extract opcode

    def get_opcode_count(self, file_path):
        opcode_dic = {}

        hex_integer = Word(hexnums) + WordEnd()  # use WordEnd to avoid parsing leading a-f of non-hex numbers as a hex
        line = ".text:" + hex_integer + Optional(
            (hex_integer * (1,))("instructions") + Word(alphas, alphanums)("opcode"))

        f = open(file_path, 'r', encoding='latin-1')
        source = f.read().splitlines()
        f.close()

        for source_line in source:
            try:
                if '.text:' in source_line:
                    result = line.parseString(source_line)
                    if "opcode" in result:
                        if result.opcode in opcode_dic:
                            opcode_dic[result.opcode] += 1
                        else:
                            opcode_dic[result.opcode] = 1
                    # print(result.opcode, result.instructions.asList())
            except:
                continue
        return opcode_dic

    # get section count
    def get_section_count(self, file_path):
        section_dic = {}

        line = "." + Word(alphas, alphanums)("section")

        f = open(file_path, 'r', encoding='latin-1')
        source = f.read().splitlines()
        f.close()

        for source_line in source:
            try:
                result = line.parseString(source_line)
                if "section" in result:
                    if result.section in section_dic:
                        section_dic[result.section] += 1
                    else:
                        section_dic[result.section] = 1
                    # print('section', result.section)
            except:
                continue
        return section_dic

    def feature_extraction_opcode(self, folder_path):
        total_dict = []
        for file in os.listdir(folder_path):
            if file.find('asm') != -1:
                # print('current file', file)
                # get unique count
                file_path = os.path.join(folder_path, file)
                uniq = self.get_opcode_count(file_path)

                # get class value
                label = self.classes.loc[self.classes['Id'] == file.split('.')[0]]['Class'].values[0]
                uniq['Class'] = label
                # print(len(uniq))
                # print(uniq)
                total_dict.append(uniq)

                data = pd.DataFrame(total_dict)
        return data

    def feature_extraction_section(self, folder_path):
        total_dict = []
        for file in os.listdir(folder_path):
            if file.find('asm') != -1:
                # print('current file', file)
                # get unique count
                file_path = os.path.join(folder_path, file)
                uniq = self.get_section_count(file_path)

                # get class value
                label = self.classes.loc[self.classes['Id'] == file.split('.')[0]]['Class'].values[0]
                uniq['Class'] = label
                # print(len(uniq))
                # print(uniq)
                total_dict.append(uniq)

                data = pd.DataFrame(total_dict)
        return data

**MODEL CLASS**

In [13]:
#ignore warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#import the necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, Reshape, Dropout
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt

class Model:
    def __init_subclass__(cls) -> None:
        pass

    def decisionTree(self, X_train, y_train, X_test, y_test):
        # training a DescisionTreeClassifier
        print('DescisionTreeClassifier')

        # dtree_model = DecisionTreeClassifier(criterion="entropy", max_depth=3).fit(X_train, y_train)
        dtree_model = DecisionTreeClassifier(criterion="gini", max_depth=3).fit(X_train, y_train)
        dtree_predictions = dtree_model.predict(X_test)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, dtree_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, dtree_predictions,
                                                                               average='weighted')
        cm = confusion_matrix(y_test, dtree_predictions)
        print(cm)

        print("precision_recall_fscore_support:", precision)
        print("tpr = recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, dtree_predictions)
        return accuracy, precision, recall, fscore, fpr

    # training a linear SVM classifier
    def SVClassifier(self, X_train, y_train, X_test, y_test):
        print('SVC Classifier')
        svm_model_linear = SVC(kernel='linear', C=1).fit(X_train, y_train)
        svm_predictions = svm_model_linear.predict(X_test)

        # creating a confusion matrix
        # cm = confusion_matrix(y_test, svm_predictions)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, svm_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, svm_predictions,
                                                                               average='weighted')
        
        cm = confusion_matrix(y_test, svm_predictions)
        print(cm)
        
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, svm_predictions)
        return accuracy, precision, recall, fscore, fpr

    # training a KNN classifier
    def KNearestClassifier(self, X_train, y_train, X_test, y_test):
        print('KNearestClassifier')
        knn = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)

        # prediction matrix
        knn_predictions = knn.predict(X_test)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, knn_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, knn_predictions,
                                                                               average='weighted')
        cm = confusion_matrix(y_test, knn_predictions)
        print(cm)
        
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, knn_predictions)
        return accuracy, precision, recall, fscore, fpr

    # training a Naive Bayes classifier
    def NaiveBayes(self, X_train, y_train, X_test, y_test):
        print('NaiveBayes')
        gnb = GaussianNB().fit(X_train, y_train)
        gnb_predictions = gnb.predict(X_test)

        # accuracy on X_test
        accuracy = gnb.score(X_test, y_test)
        print(accuracy)

        # creating a confusion matrix
        cm = confusion_matrix(y_test, gnb_predictions)

        # Metrics evaluation
        accuracy = metrics.accuracy_score(y_test, gnb_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, gnb_predictions,
                                                                               average='weighted')
        cm = confusion_matrix(y_test, gnb_predictions)
        print(cm)
        
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, gnb_predictions)
        return accuracy, precision, recall, fscore, fpr

    def SGD(self, X_train, y_train, X_test, y_test):
        print('SGD')
        clf = SGDClassifier(alpha=0.001, max_iter=100).fit(X_train, y_train)
        clf_predictions = clf.predict(X_test)

        # Model evaluation
        accuracy = metrics.accuracy_score(y_test, clf_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, clf_predictions,
                                                                               average='weighted')
        cm = confusion_matrix(y_test, clf_predictions)
        print(cm)
        
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, clf_predictions)
        return accuracy, precision, recall, fscore, fpr

    def LogisticReg(self, X_train, y_train, X_test, y_test):
        print('Logistic Regression')
        clf = LogisticRegression(random_state=0).fit(X_train, y_train)
        clf_predictions = clf.predict(X_test)

        # Model evaluation
        accuracy = metrics.accuracy_score(y_test, clf_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, clf_predictions,
                                                                               average='weighted')
        cm = confusion_matrix(y_test, clf_predictions)
        print(cm)
        
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, clf_predictions)
        return accuracy, precision, recall, fscore, fpr

    def RandomForest(self, X_train, y_train, X_test, y_test):
        print('RandomForest')
        clf = RandomForestClassifier(max_depth=2, random_state=0)
        clf.fit(X_train, y_train)
        clf_predictions = clf.predict(X_test)

        # Model evaluation
        accuracy = metrics.accuracy_score(y_test, clf_predictions)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_test, clf_predictions,
                                                                               average='weighted')
        cm = confusion_matrix(y_test, clf_predictions)
        print(cm)
        
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        fpr = self.calculate_tpr_fpr(y_test, clf_predictions)
        return accuracy, precision, recall, fscore, fpr

    def DnnPreprocessing(self, X, Y):
        X = np.array(X)

        # Transform name species into numerical values
        encoder = LabelEncoder()
        encoder.fit(Y)
        Y = encoder.transform(Y)
        Y = np_utils.to_categorical(Y)
        
        print("Before oversampling",  X.shape, Y.shape)
        
        oversample = RandomOverSampler(sampling_strategy="not majority")  #SMOTE()
        
        X, Y = oversample.fit_resample(X, Y)
        
        print("After oversampling",  X.shape, Y.shape)

        # split the data into train and test
        train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1, random_state=0)
        return train_x, test_x, train_y, test_y, encoder
    
    

    def DNN_Model(self, train_x, test_x, train_y, test_y, epoch, encoder):
        
        print("dim: ", train_x.shape[1], train_y.shape[1])
        
        model = Sequential()
        model.add(Dense(30, input_dim=train_x.shape[1], activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(15, activation = "relu"))
        model.add(Dropout(0.2))
        model.add(Dense(9, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        history = model.fit(train_x, train_y, epochs=epoch, batch_size=100, validation_data=(test_x, test_y))

        scores = model.evaluate(test_x, test_y)
        print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

        predictions = model.predict(test_x)
        prediction_ = np.argmax(predictions, axis=1)
        prediction_ = encoder.inverse_transform(prediction_)

        test_y_ = np.argmax(test_y, axis=1)
        test_y_ = encoder.inverse_transform(test_y_)

        # Model evaluation
        accuracy = metrics.accuracy_score(test_y_, prediction_)
        print("Accuracy:", accuracy)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(test_y_, prediction_, average='weighted')
        print("precision_recall_fscore_support:", precision)
        print("recall:", recall)
        print("fscore:", fscore)
        self.plot_Acc(history, epoch)
        fpr = self.calculate_tpr_fpr(test_y_, prediction_)
        return accuracy, precision, recall, fscore, fpr

    def plot_Acc(self, history, epoch):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']

        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs_range = range(epoch)

        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.legend(loc='lower right')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.legend(loc='upper right')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()

    def calculate_tpr_fpr(self, true_y, pred_y):
        # creating a confusion matrix
        confusion_matrix = metrics.confusion_matrix(true_y, pred_y)

        FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
        FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
        TP = np.diag(confusion_matrix)
        TN = confusion_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        # Sensitivity, hit rate, recall, or true positive rate
        # TPR = TP / (TP + FN)
        # Fall out or false positive rate
        FPR = FP / (FP + TN)
        print('FPR', np.average(FPR))
        return np.average(FPR)

**Main**

In [4]:
# importing necessary libraries
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np # linear algebra

# from feature_extractor import FeatureExtraction
# from models import Model

data_path = "malware-classification"


trainLabel_path = os.path.join(data_path, 'trainLabels.csv')
MalwaDataPath = os.path.join(data_path, 'bytes')
source = os.path.join(data_path, 'asmFiles')

#trainLabel_path = 'trainLabels.csv'
#MalwaDataPath = r'C:\Users\georg\OneDrive\Desktop\Malware Classification - deep learning\dataSample'

In [8]:
data_path = "malware-classification"

#source = os.path.join(data_path, 'train') 
byte_destination = os.path.join(data_path, 'byteFiles')
source = os.path.join(data_path, 'asmFiles')


if not os.path.isdir(byte_destination):
    os.makedirs(byte_destination)

if os.path.isdir(source):
    #os.rename(source, asm_destination)
    #source = asm_destination
    all_files = os.listdir(source)
    for file in all_files:
        if (file.endswith("bytes")):
            shutil.move( os.path.join(asm_destination, file), byte_destination)

In [12]:
files = os.listdir(byte_destination)
files2 = os.listdir(source)

for file in files[:5857]:
    os.remove(os.path.join(byte_destination, file))
    
for file in files2[:5857]:
    os.remove(os.path.join(source, file))

In [14]:
FE = FeatureExtraction(trainLabel_path)

data_byte = FE.feature_extraction_bytes(byte_destination)

data_byte.to_csv('file_byte.csv', encoding='utf-8', index=False)
print (data_byte.head())

        00      97      FF      C6      49      C3      63      C0      74  \
0  35208.0  1545.0  5996.0   508.0   873.0   741.0   383.0  1056.0  1803.0   
1  11674.0  3383.0  6864.0  3243.0  3185.0  3169.0  3417.0  3329.0  3402.0   
2  21075.0   510.0  8366.0   586.0   689.0   669.0   430.0  1153.0  1843.0   
3  21689.0    63.0  8769.0  1962.0   305.0  2224.0   432.0  1080.0  1820.0   
4   5743.0  3022.0  3163.0  3006.0  2973.0  3144.0  2977.0  3031.0  3053.0   

       59  ...      4B      DF      B1      EA      9D      A3      DE  \
0   958.0  ...   311.0   285.0   282.0   315.0   281.0   244.0   274.0   
1  3327.0  ...  3275.0  3244.0  3248.0  3221.0  3213.0  3187.0  3134.0   
2   506.0  ...   386.0   320.0   304.0   328.0   502.0   375.0   335.0   
3   925.0  ...   198.0   884.0    62.0  2462.0   858.0   141.0   124.0   
4  3081.0  ...  3044.0  3043.0  3009.0  3055.0  3034.0  3083.0  3058.0   

       B9         ??  Class  
0   274.0      364.0      9  
1  3281.0  1491612.0      

In [7]:
FE = FeatureExtraction(trainLabel_path)

data_section = FE.feature_extraction_section(source)

data_section.to_csv('file_section.csv', encoding='utf-8', index=False)
print (data_section.head())

      text  idata    rdata      data  Class  rsrc  reloc  rata  bss  code  \
0  10290.0  241.0   1265.0     495.0      9   NaN    NaN   NaN  NaN   NaN   
1   1694.0  126.0    211.0    1069.0      3   3.0    NaN   NaN  NaN   NaN   
2  31295.0  379.0      NaN    1029.0      1   3.0    3.0   NaN  NaN   NaN   
3  91180.0  507.0  21277.0    6551.0      2   NaN    NaN   NaN  NaN   NaN   
4   1141.0  120.0    658.0  783880.0      3   NaN    NaN   NaN  NaN   NaN   

   ...  debug01  cdata  bdata  bine7  opdw7  rema7  lo7c  samp7  welc7  musy7  
0  ...      NaN    NaN    NaN    NaN    NaN    NaN   NaN    NaN    NaN    NaN  
1  ...      NaN    NaN    NaN    NaN    NaN    NaN   NaN    NaN    NaN    NaN  
2  ...      NaN    NaN    NaN    NaN    NaN    NaN   NaN    NaN    NaN    NaN  
3  ...      NaN    NaN    NaN    NaN    NaN    NaN   NaN    NaN    NaN    NaN  
4  ...      NaN    NaN    NaN    NaN    NaN    NaN   NaN    NaN    NaN    NaN  

[5 rows x 184 columns]


In [5]:
FE = FeatureExtraction(trainLabel_path)

data_opcode = FE.feature_extraction_opcode(source)

data_opcode.to_csv('file_opcode.csv', encoding='utf-8', index=False)
print (data_opcode.head())

      FF    F8     E8   A1    EB    E0    C3   DB    E9    FE  ...  \
0  376.0  41.0  154.0  9.0  34.0  12.0  44.0  5.0  14.0  39.0  ...   
1    NaN   NaN    NaN  2.0   1.0   NaN   NaN  NaN   NaN   1.0  ...   
2    1.0   NaN    NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   
3    NaN   NaN    NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...   
4    NaN   NaN    1.0  NaN   NaN   1.0   1.0  NaN   NaN   2.0  ...   

   TempFileName  DateStr  aGGGGjgGhgtg  aGGGGagvgGag  aGGGGG3gGpg  aGsg3g  \
0           NaN      NaN           NaN           NaN          NaN     NaN   
1           NaN      NaN           NaN           NaN          NaN     NaN   
2           NaN      NaN           NaN           NaN          NaN     NaN   
3           NaN      NaN           NaN           NaN          NaN     NaN   
4           NaN      NaN           NaN           NaN          NaN     NaN   

   aGhg9g  aXx88  aMfcsubs  aRtm  
0     NaN    NaN       NaN   NaN  
1     NaN    NaN       NaN   NaN  
2     NaN  

In [6]:
df1 = pd.read_csv("file_byte.csv", sep=",", index_col=False)
df2 = pd.read_csv("file_section.csv", sep=",", index_col=False)

if 'Class' in df1.columns:
    print('true')
    
if 'Class' in df2.columns:
    df2 = df2.drop('Class', axis=1)
    


new_df = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)

true


In [7]:
new_df.shape

(5000, 441)

In [8]:
df3 =  pd.read_csv("file_opcode.csv", sep=",", index_col=False)

if 'Class' in df3.columns:
    df3 = df3.drop('Class', axis=1)

final_df = pd.concat([new_df.reset_index(drop=True), df3.reset_index(drop=True)], axis=1)

In [9]:
final_df.head(10)

,00,97,FF,C6,49,C3,63,C0,74,59,...,TempFileName,DateStr,aGGGGjgGhgtg,aGGGGagvgGag,aGGGGG3gGpg,aGsg3g,aGhg9g,aXx88,aMfcsubs,aRtm
0,35208.0,1545.0,5996.0,508.0,873.0,741.0,383.0,1056.0,1803.0,958.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11674.0,3383.0,6864.0,3243.0,3185.0,3169.0,3417.0,3329.0,3402.0,3327.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21075.0,510.0,8366.0,586.0,689.0,669.0,430.0,1153.0,1843.0,506.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21689.0,63.0,8769.0,1962.0,305.0,2224.0,432.0,1080.0,1820.0,925.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5743.0,3022.0,3163.0,3006.0,2973.0,3144.0,2977.0,3031.0,3053.0,3081.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,88264.0,342.0,2701.0,510.0,451.0,509.0,624.0,918.0,1441.0,407.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7539.0,3166.0,5020.0,3101.0,3171.0,3180.0,3058.0,3096.0,3140.0,3130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11409.0,3208.0,6766.0,3120.0,3298.0,3208.0,3410.0,3262.0,3335.0,3263.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11522.0,3250.0,7120.0,3138.0,3232.0,3281.0,3255.0,3314.0,3282.0,3213.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,71800.0,1692.0,4917.0,1613.0,1883.0,1723.0,3627.0,1981.0,5120.0,2044.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

print('=========================================================================================================================')
print('RUNNING FOR OPCODES')
metrics_list_opcodes = []
model_labels=[]

FE = FeatureExtraction('trainLabels.csv')
# data = FE.feature_extraction_bytes(MalwaDataPath)
data = FE.feature_extraction_opcode(MalwaDataPath)
# data = FE.feature_extraction_section('MalwareData')
print(data.head())


# X -> features, y -> label
X = data.loc[:, data.columns != 'Class']
X = np.array(X)
print('is nan', np.any(np.isnan(X)))
X = np.nan_to_num(X)
print('is nan', np.any(np.isnan(X)))

y = data['Class']


# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)



my_models = Model()
# accuracy, precision, recall, fscore, fpr = my_models.decisionTree(X_train, y_train, X_test, y_test)
model_labels.append('DT')
metrics_list_opcodes.append(my_models.decisionTree(X_train, y_train, X_test, y_test))

model_labels.append('SVC')
metrics_list_opcodes.append(my_models.SVClassifier(X_train, y_train, X_test, y_test))

model_labels.append('KNN')
metrics_list_opcodes.append(my_models.KNearestClassifier(X_train, y_train, X_test, y_test))

model_labels.append('NB')
metrics_list_opcodes.append(my_models.NaiveBayes(X_train, y_train, X_test, y_test))

model_labels.append('SGD')
metrics_list_opcodes.append(my_models.SGD(X_train, y_train, X_test, y_test))

model_labels.append('LR')
metrics_list_opcodes.append(my_models.LogisticReg(X_train, y_train, X_test, y_test))

model_labels.append('RF')
metrics_list_opcodes.append(my_models.RandomForest(X_train, y_train, X_test, y_test))


train_x, test_x, train_y, test_y, encoder = my_models.DnnPreprocessing(X, y)
accuracy, precision, recall, fscore, fpr = my_models.DNN_Model(train_x, test_x, train_y, test_y, epoch=100, encoder=encoder)

model_labels.append('DNN')
metrics_list_opcodes.append((accuracy, precision, recall, fscore, fpr))


print('=================================================================================================================================')



In [ ]:

print('=========================================================================================================================')
print('RUNNING FOR SECTION')
metrics_list_section = []
model_labels=[]

FE = FeatureExtraction('trainLabels.csv')
# data = FE.feature_extraction_bytes(MalwaDataPath)
# data = FE.feature_extraction_opcode(MalwaDataPath)
data = FE.feature_extraction_section(MalwaDataPath)
print(data.head())


# X -> features, y -> label
X = data.loc[:, data.columns != 'Class']
X = np.array(X)
print('is nan', np.any(np.isnan(X)))
X = np.nan_to_num(X)
print('is nan', np.any(np.isnan(X)))

y = data['Class']


# dividing X, y into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)



my_models = Model()
# accuracy, precision, recall, fscore, fpr = my_models.decisionTree(X_train, y_train, X_test, y_test)
model_labels.append('DT')
metrics_list_section.append(my_models.decisionTree(X_train, y_train, X_test, y_test))

model_labels.append('SVC')
metrics_list_section.append(my_models.SVClassifier(X_train, y_train, X_test, y_test))

model_labels.append('KNN')
metrics_list_section.append(my_models.KNearestClassifier(X_train, y_train, X_test, y_test))

model_labels.append('NB')
metrics_list_section.append(my_models.NaiveBayes(X_train, y_train, X_test, y_test))

model_labels.append('SGD')
metrics_list_section.append(my_models.SGD(X_train, y_train, X_test, y_test))

model_labels.append('LR')
metrics_list_section.append(my_models.LogisticReg(X_train, y_train, X_test, y_test))

model_labels.append('RF')
metrics_list_section.append(my_models.RandomForest(X_train, y_train, X_test, y_test))

train_x, test_x, train_y, test_y, encoder = my_models.DnnPreprocessing(X, y)
accuracy, precision, recall, fscore, fpr = my_models.DNN_Model(train_x, test_x, train_y, test_y, epoch=100, encoder=encoder)

model_labels.append('DNN')
metrics_list_section.append((accuracy, precision, recall, fscore, fpr))


print('=================================================================================================================================')



**Analysis**

In [ ]:

#Analysis
print('ANALYSIS')
accuracies = [a[0] for a in metrics_list_bytes]
accuracies2 = [a[0] for a in metrics_list_opcodes]
accuracies3 = [a[0] for a in metrics_list_section]
plt.plot(model_labels, accuracies, '-o', label='bytecodes')
plt.plot(model_labels, accuracies2, '-o', label='opcodes')
plt.plot(model_labels, accuracies3, '-o', label='section')
plt.legend()
plt.title('Accuracy')
plt.show()


tpr = [a[2] for a in metrics_list_bytes]
fpr = [a[4] for a in metrics_list_bytes]
x = np.arange(len(model_labels))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, tpr, width, label='tpr_byte')
rects2 = ax.bar(x + width/2, fpr, width, label='fpr_byte')
# Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_ylabel('Scores')
ax.set_title('TPR and FPR bytes')
# ax.set_xticks(x, model_labels)
ax.legend()
ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
fig.tight_layout()
plt.show()


tpr = [a[2] for a in metrics_list_opcodes]
fpr = [a[4] for a in metrics_list_opcodes]
x = np.arange(len(model_labels))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, tpr, width, label='tpr_opcode')
rects2 = ax.bar(x + width/2, fpr, width, label='fpr_opcode')
# Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_ylabel('Scores')
ax.set_title('TPR and FPR opcodes')
# ax.set_xticks(x, model_labels)
ax.legend()
ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
fig.tight_layout()
plt.show()


tpr = [a[2] for a in metrics_list_section]
fpr = [a[4] for a in metrics_list_section]
x = np.arange(len(model_labels))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, tpr, width, label='tpr_section')
rects2 = ax.bar(x + width/2, fpr, width, label='fpr_section')
# Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_ylabel('Scores')
ax.set_title('TPR and FPR section')
# ax.set_xticks(x, model_labels)
ax.legend()
ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
fig.tight_layout()
plt.show()